In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame


weather_data_to_load = "output_data_cities.csv"
weather_df = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_df.dropna()
dropna_weather_data.head(20)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,67,hambantota,40,LK,1558378766,100,6.12,81.12,80.60,3.36
1,159,manado,40,ID,1558378784,100,1.49,124.85,73.40,4.97
2,177,keflavik,75,IS,1558378788,100,64.00,-22.56,51.01,9.17
3,212,mount gambier,90,AU,1558378628,100,-37.83,140.78,53.60,8.05
4,243,buchach,75,UA,1558378822,100,49.06,25.39,70.00,2.24
5,283,christchurch,1,NZ,1558378744,100,-43.53,172.64,51.01,3.36
6,289,union,90,US,1558378842,100,47.36,-123.10,57.20,8.05
7,358,alyangula,100,AU,1558378858,100,-13.85,136.42,71.60,3.36
8,372,shimoda,75,JP,1558378860,100,34.70,138.93,68.00,11.41
9,398,grindavik,75,IS,1558378871,100,63.84,-22.43,51.01,9.17


In [3]:

# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.



# Store latitude and longitude in locations
lat_lon = weather_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humid = weather_df["Humidity"]
#lat_lon.head()

In [4]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df.Max_Temp < 80) & (weather_df.Max_Temp > 70)
crit_windSpeed = weather_df.Wind_Speed < 15
crit_cloudiness = weather_df.Cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)


,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,105,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
1,296,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
2,301,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
3,318,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
4,386,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
5,418,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
6,474,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
7,512,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
8,518,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [6]:
# Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
#hotel_df.head()



In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#hotel_df.info()

In [8]:

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
#<dt>Country</dt><dd>{Country}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…